# 03. Generate Puzzle Sheets

This notebook generates puzzle sheets with ArUco markers and saves them to a tagged directory in the data folder.


## 1. Import Libraries and SnapFitPaths


In [ ]:
import json
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np

from snap_fit.aruco.aruco_board import ArucoBoardGenerator
from snap_fit.config.aruco.aruco_board_config import ArucoBoardConfig
from snap_fit.params.snap_fit_params import get_snap_fit_paths
from snap_fit.params.snap_fit_paths import SnapFitPaths
from snap_fit.puzzle import PieceStyle
from snap_fit.puzzle import PuzzleConfig
from snap_fit.puzzle import PuzzleGenerator
from snap_fit.puzzle import PuzzleRasterizer
from snap_fit.puzzle import PuzzleSheetComposer
from snap_fit.puzzle import SheetLayout

## 2. Define Global Configurations

Consolidate all configuration objects here.


In [ ]:
# Generation Tag
tag_name = "sample_puzzle_v1"

# Puzzle configuration
puzzle_config = PuzzleConfig(
    width=200 * 3,
    height=150 * 3,
    tiles_x=8,
    tiles_y=6,
    seed=42,
    tab_size=0.2,
    jitter=0.14,
    corner_radius=1.5,
    font_size=3,
)

# ArUco board configuration
aruco_config = ArucoBoardConfig(
    markers_x=7,
    markers_y=5,
    marker_length=100,
    marker_separation=100,
)

# Sheet layout configuration
sheet_layout = SheetLayout(
    sheet_width=297,
    sheet_height=210,
    margin=40,
    piece_spacing=100,
    dpi=300,
)

# Piece style for the sheets
sheet_style = PieceStyle(
    fill="black",
    stroke="red",
    stroke_width=0,
    label_color="white",
)

print(f"Tag: {tag_name}")
print(f"Puzzle: {puzzle_config.tiles_x}x{puzzle_config.tiles_y} pieces")
print(
    f"Sheet: {sheet_layout.sheet_width}x{sheet_layout.sheet_height} mm"
    f" at {sheet_layout.dpi} DPI"
)

## 3. Initialize Output Paths


In [ ]:
paths = get_snap_fit_paths()
output_dir = paths.data_fol / tag_name
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Output directory: {output_dir}")

## 4. Generate Puzzle and ArUco Board


In [ ]:
# Generate puzzle pieces
generator = PuzzleGenerator(puzzle_config)
pieces = generator.generate()

# Generate ArUco board background
aruco_generator = ArucoBoardGenerator(aruco_config)
aruco_img = aruco_generator.generate_image()

# Convert to BGR for OpenCV consistency
if len(aruco_img.shape) == 2:  # noqa: PLR2004
    aruco_img_bgr = cv2.cvtColor(aruco_img, cv2.COLOR_GRAY2BGR)
else:
    aruco_img_bgr = aruco_img

print(f"Generated {len(pieces)} pieces")
print(f"ArUco board image shape: {aruco_img_bgr.shape}")

## 5. Compose and Generate Sheets


In [ ]:
composer = PuzzleSheetComposer(sheet_layout, aruco_img_bgr, piece_style=sheet_style)
sheets = composer.generate_all_sheets(generator)

print(f"Generated {len(sheets)} sheets")


In [ ]:
# Display the first sheet as a preview
if len(sheets) > 0:
    plt.figure(figsize=(12, 8))
    plt.imshow(sheets[0][:, :, ::-1])
    plt.axis("off")
    plt.title(f"Preview: Sheet 1/{len(sheets)}")
    plt.show()

## 6. Save Output Files and Configuration JSON


In [ ]:
# Save sheets
sheet_paths = composer.save_sheets(sheets, str(output_dir), prefix=f"{tag_name}_sheet")
print(f"Saved {len(sheet_paths)} sheets to {output_dir}")

# Save full puzzle SVG for reference
puzzle_svg = generator.to_svg(include_labels=True)
(output_dir / "puzzle_full.svg").write_text(puzzle_svg)

# Dump all configs as JSON
config_data = {
    "tag_name": tag_name,
    "puzzle_config": puzzle_config.model_dump(),
    "aruco_config": aruco_config.model_dump(),
    "sheet_layout": sheet_layout.model_dump(),
    "sheet_style": sheet_style.model_dump(),
}
config_path = output_dir / "config.json"
config_str = json.dumps(config_data, indent=4)
config_path.write_text(config_str)
print(f"Saved configuration to {config_path}")